# THn visualization with RootInteractive
### Import libraries

* colorcet optional (for more ROOT colz like color pallete import it and use cc.b_linear_kbgoy_20_95_c57)

In [ ]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
import base64
import ctypes
import logging
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
import colorcet as cc

### Load the THnSparse file

In [ ]:
myFile = ROOT.TFile.Open("~/Software/TPC_QC_Visualization/Data/UserFiles/myFile.root")
hist = myFile.ExpertVis
thn = hist.Get("ExpertVis;1")
dimensions = []
for i in range(thn.GetNdimensions()):
    dimensions.append(thn.GetAxis(i).GetTitle())
thn.ls()

### Convert THnSparse to pandas.DataFrame
* the binning parameters should be known (ToDo: investigate if this can be extracted from the THn) 

In [ ]:
bin_count = [40, 100, 100, 72, 24, 24, 40, 3]
bin_min = [0.05, 1., 1., 0., -1.2, -1.2, 52., -1.]
bin_max = [20., 2000., 1000., 6.3, 1.2, 1.2, 212., 1.]
dim_count = len(bin_count)
bin_width = [(bin_max[x]-bin_min[x])/(bin_count[x]+1) for x in range(dim_count)]

tracks = [];
coord = (ctypes.c_int*dim_count)()
#for Bin in range(thn.GetNbins()):
for Bin in range(100000):
    val = thn.GetBinContent(Bin,coord)
    tracks.append(val)
    for Dim in range(dim_count):
        #tracks.append(bin_min[Dim]+(coord[Dim]-0.5)*bin_width[Dim])
        tracks.append(coord[Dim])
    
df = pd.DataFrame(np.reshape(tracks, newshape=(int(len(tracks)/(dim_count+1)), dim_count+1)), 
                         columns = ['Weight','P','dEdxTot','dEdxMax','Phi','Tgl','Snp','NClusters','Sign'])
initMetadata(df)
df.head()

## Plot as scatter plot
Doesn't make too much sense for us here since the data from all the dimensions are not aggregated

In [ ]:
output_file("thn_scatter.html")

figureArray = [
    [['P'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['NClusters'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Phi'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Tgl'], ['dEdxTot'], {"colorZvar": "Weight"}],
    {"size":2, "legend_options": {"label_text_font_size": "13px"}}
]
layout = {
    "dEdx (a.u.) vs p (GeV/c)": [
        [0, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs ncls": [
        [1, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs φ (rad)": [
        [2, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs tanλ": [
        [3, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]]
}
tooltips = []
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'sizing_mode': 'scale_width'}]
    }    
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=layout,
                      tooltips=tooltips, widgetLayout=widgetLayoutDesc)

## As 1D histograms
Makes sense

In [ ]:
output_file("thn_1D.html")

histoArray = [
    {"name": "dEdxTot_hist", "variables": ["dEdxTot"], "nbins": 100, "weights": "Weight"},
    {"name": "NClusters_hist", "variables": ["NClusters"], "nbins": 40, "weights": "Weight"},
    {"name": "Phi_hist", "variables": ["Phi"], "nbins": 72, "weights": "Weight"},
    {"name": "Tgl_hist", "variables": ["Tgl"], "nbins": 24, "weights": "Weight"},
    {"name": "P_hist", "variables": ["P"], "nbins": 40, "weights": "Weight"},
    {"name": "dEdxMax_hist", "variables": ["dEdxMax"], "nbins": 100, "weights": "Weight"}
    
]
figureArray = [
    [['dEdxTot'],['dEdxTot_hist'],{'size': 5}],
    [['NClusters'],['NClusters_hist'],{'size': 5}],
    [['Phi'],['Phi_hist'],{'size': 5}],
    [['Tgl'],['Tgl_hist'],{'size': 5}],
    [['P'],['P_hist'],{'size': 5}],
    [['dEdxMax'],['dEdxMax_hist'],{'size': 5}]
]
figureLayoutDesc=[
    [[0, 1], [2,3], [4,5], {'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width'}
]
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'width':1}]
    }
tooltips = []
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, plot_width= 500, histogramArray=histoArray)

## As 2D histograms
Also makes sense

In [ ]:
output_file("thn_2D.html")

histoArray = [
    {"name": "dEdx_vs_P", "variables": ["P", "dEdxMax"], "nbins": [40, 100], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_ncls", "variables": ["NClusters", "dEdxTot"], "nbins": [40, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_phi", "variables": ["Phi", "dEdxTot"], "nbins": [72, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_tgl", "variables": ["Tgl", "dEdxTot"], "nbins": [24, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []}
]
figureArray = [
    [['P'], ['dEdx_vs_P']],
    [['NClusters'], ['dEdx_vs_ncls']],
    [['Phi'], ['dEdx_vs_phi']],
    [['Tgl'], ['dEdx_vs_tgl']],
    ["tableHisto", {"rowwise": True}]
]
figureLayoutDesc={
    "dEdx_vs_p" : [[0,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_ncls" : [[1,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_phi" : [[2,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}],
    "dEdx_vs_tgl" : [[3,4, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}],
        {'plot_height': 400, 'sizing_mode': 'scale_width', 'y_visible' : 2}]
}
tooltips = []
widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'width':1}]
    }
xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, histogramArray=histoArray, plot_width= 500,
                            colors="Bokeh",rescaleColorMapper=True, nCols = 4)